In [ ]:
# 데이터베이스 자동 구축을 위한 라이브러리 
#!pip install schedule

In [ ]:
import pandas as pd
import numpy as np
import schedule
import time
import re

from bs4 import BeautifulSoup
from datetime import datetime
from tqdm import tqdm
import requests

# path를 자신의 경로에 맞게 지정!
path = "/Users/Goo/대구_빅데이터/"

# Caution
- 기존 데이터베이스 없이 처음 자동화를 시작하려고 한다면 아래의 코드를 실행하여 path에 소비_Daily_0.csv ~ 소비_Daily_23.csv를 구축하여 주세요. 아래의 코드는 데이터베이스가 없는 경우 1회만 작동시키면 됩니다.
- 서비스 상용화 하루전은 데이터베이스를 구축하는 시간이 필요합니다.

In [ ]:
# columns = ["company", "url", "title", "content", "label"]
# daily_df = pd.DataFrame(columns=columns)

# for time in range(0, 24):
#     daily_df.to_csv(path + f"소비_Daily_{time}.csv", index=False)

## Daily news data 자동화 코드 목적
뉴스 추천을 받는 사람이 CAUsumer의 서비스를 활용한 시점으로부터 만 하루의 뉴스 데이터를 추천 받도록하기 위해 1시간 간격으로 최신 뉴스를 업데이트하여 서비스 활용 시점으로부터 24시간전의 뉴스를 갱신하도록 코드를 자동화하였다. 예를들어 25일 저녁 6시에 5시 ~ 6시에 올라온 기사를 crawling하여 업데이트하게된다면 24일 5시 ~ 6시의 뉴스 데이터는 데이터베이스에서 사라지는 구조로 추천시점으로부터 24시간의 데이터베이스가 항상 유지되도록 구축하였다.


### Crawling 조건
URL Format : https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_content}&start={page}&pd={period}

### 조건
- search_content : 소비
- period : 7은 crawling 시점으로부터 1시간전의 뉴스를 의미함.
- page는 1, 11, 21 형식으로 페이지마다 10씩 늘어남.  
ex) 1 = 1page, 11 = 2page, 21 = 3page


Features : 신문사, 링크, 제목, 원문, label

In [ ]:
search_content = "소비"
period = 7
label = "소비"

In [ ]:
# URL 계절별 crawling 함수
# -> Redirect 되지않는 네이버뉴스 max_news개가 추출될때까지 크롤링을 계속하는 함수

news_company = []
news_title = []
news_content = []
news_url = []

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

def url_crawling():
    global news_url
    
    # 집합 형태로 저장해 중복 url 제거
    url_set = set()
    for page in tqdm(range(1, 10000, 10)):
        response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_content}&start={page}&pd={period}", headers=headers)
        # page를 넘기다 page가 없으면 종료
        # 200은 HTTP 상태코드중 하나로 OK의 의미를 가짐. 요청이 성공적으로 처리되었음을 나타냄. 200이 아니라는것은 페이지가 없어 페이지를 넘길 수 없다는 의미
        if response.status_code != 200:
            print(f"페이지 {page//10}가 없습니다. Exiting.")
            break
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        ul = soup.select_one("div.group_news > ul.list_news")

        if ul is None:
            break
        li_list = ul.find_all('li')
        for li in li_list:
            a_list = li.select('div.news_area > div.news_info > div.info_group > a.info')
            for a_tag in a_list:
                href = a_tag.get('href')
                # href 속성값이 "n.news.naver.com"(네이버 뉴스)을 포함하는지 확인한다.
                if "n.news.naver.com" in href:
                    try:
                        # request.head()로 추출한 url이 rediret되는지 확인한다. redirect 되지않은 url만 저장한다.
                        response = requests.head(href, allow_redirects=True)
                        if response.status_code == 200:
                            url_set.add(href)
                    except Exception as e:
                        print(f"An error occurred: {e}")
        time.sleep(1)

    # 신문사, 본문, 제목 추출
    for url in tqdm(url_set):
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        company = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_top > a > img[alt]")
        news_company.append(company['alt'] if company else 'None')
        title = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        news_title.append(title.text if title else 'None')
        content = soup.select_one("article#dic_area")
        news_content.append(content.text if content else 'None')
        
    news_url = list(url_set)

In [ ]:
# 시간별로 갱신하는 csv 생성 함수

def make_csv(time):
    columns = ["company", "url", "title", "content", "label"]

    data = {
        "company": news_company,
        "url": news_url,
        "title": news_title,
        "content": news_content,
        "label" : [label] * len(news_company)
    }

    daily_news = pd.DataFrame(data, columns=columns)
    daily_news = daily_news.drop_duplicates(subset=['content'], keep='first')
    daily_news.to_csv(path+f"{label}_Daily_{time}.csv", index=False)

In [ ]:
# 24시간 데이터베이스를 유지하는 최종.csv를 생성하는 함수

def final_csv():

    file_names = [f"소비_Daily_{time}.csv" for time in range(0, 24)]
    dfs = [pd.read_csv(path + file) for file in file_names]
    final_df = pd.concat(dfs, ignore_index=True)
    
    final_df.to_csv(path+"daily_data.csv", index=False)

In [ ]:
# schedule을 사용하기 위해 모든 함수들을 실행하는 함수

def make_time_dailynews(time):
    # 1시간 간격으로 실시간 뉴스를 크롤링해올때, 리스트를 초기화하여 해당 시간의 뉴스만을 가져오기 위해 리스트를 초기화 하여줌
    global news_company, news_title, news_content, news_url

    # 리스트 초기화
    news_company = []
    news_title = []
    news_content = []
    news_url = []
    
    url_crawling()
    make_csv(time)
    final_csv()

## Caution
- 만약, AttributeError : 'int' object has no attribute 'sleep'을 만나게 된다면 위로 올라가 라이브러리들을 다시 실행시켜주세요!  
- import time이 실행되지 않아 발생하는 오류로 추정중입니다.

In [ ]:
# 자동화 함수 구현

def schedule_make_time_dailynews(time):
    def wrapper():
        make_time_dailynews(time)
    return wrapper

# 스케줄 등록 : 지정된 시각(1시간 간격)으로 뉴스를 크롤링 해오도록 자동화시스템 구현
schedule.every().day.at("00:00").do(schedule_make_time_dailynews(0))
schedule.every().day.at("01:00").do(schedule_make_time_dailynews(1))
schedule.every().day.at("02:00").do(schedule_make_time_dailynews(2))
schedule.every().day.at("03:00").do(schedule_make_time_dailynews(3))
schedule.every().day.at("04:00").do(schedule_make_time_dailynews(4))
schedule.every().day.at("05:00").do(schedule_make_time_dailynews(5))
schedule.every().day.at("06:00").do(schedule_make_time_dailynews(6))
schedule.every().day.at("07:00").do(schedule_make_time_dailynews(7))
schedule.every().day.at("08:00").do(schedule_make_time_dailynews(8))
schedule.every().day.at("09:00").do(schedule_make_time_dailynews(9))
schedule.every().day.at("10:00").do(schedule_make_time_dailynews(10))
schedule.every().day.at("11:00").do(schedule_make_time_dailynews(11))
schedule.every().day.at("12:00").do(schedule_make_time_dailynews(12))
schedule.every().day.at("13:00").do(schedule_make_time_dailynews(13))
schedule.every().day.at("14:00").do(schedule_make_time_dailynews(14))
schedule.every().day.at("15:00").do(schedule_make_time_dailynews(15))
schedule.every().day.at("16:00").do(schedule_make_time_dailynews(16))
schedule.every().day.at("17:00").do(schedule_make_time_dailynews(17))
schedule.every().day.at("18:00").do(schedule_make_time_dailynews(18))
schedule.every().day.at("19:00").do(schedule_make_time_dailynews(19))
schedule.every().day.at("20:00").do(schedule_make_time_dailynews(20))
schedule.every().day.at("21:00").do(schedule_make_time_dailynews(21))
schedule.every().day.at("22:00").do(schedule_make_time_dailynews(22))
schedule.every().day.at("23:00").do(schedule_make_time_dailynews(23))


while True:
    schedule.run_pending()
    time.sleep(50)